# 04 Shocks and models
This notebook inspects shock series and regression outputs.
We only run model estimation if outputs are missing.

Shock notes:
- Macro shock uses BCRP annual PBI growth (PM04863AA) as a proxy.
- Climate shock uses NOAA ONI (national index) replicated across districts
  when CHIRPS is not used.

In [ ]:
from pathlib import Path
import subprocess
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path("..").resolve()
CONFIG = ROOT / "config.yaml"
PYTHON = ROOT / ".venv" / "bin" / "python"
AUTO_RUN = False

try:
    from IPython import get_ipython

    _IN_NOTEBOOK = get_ipython() is not None
except Exception:
    _IN_NOTEBOOK = False

if not _IN_NOTEBOOK:
    import matplotlib

    matplotlib.use("Agg")


def run_cmd(args):
    subprocess.run(args, check=True, cwd=ROOT)


def ensure_artifacts(paths, auto_run=True):
    missing = [p for p in paths if not p.exists()]
    if missing and auto_run:
        run_cmd([str(PYTHON), "-m", "run", "model", "panel_shocks", "--config", str(CONFIG)])
    return missing


shock_macro = ROOT / "data" / "processed" / "shock_macro.parquet"
shock_clima = ROOT / "data" / "processed" / "shock_clima.parquet"
reg_path = ROOT / "outputs" / "tables" / "regresiones_principales.csv"
ensure_artifacts([shock_macro, shock_clima, reg_path], auto_run=AUTO_RUN)

## Macro shock series

In [ ]:
macro = pd.read_parquet(shock_macro)
macro.head()

In [ ]:
plt.plot(macro["anio"], macro["dlog_tot"])
plt.title("dlog_tot")
plt.xlabel("anio")
plt.show()

## Climate shock series

In [ ]:
clima = pd.read_parquet(shock_clima)
clima.head()

## Panel regression results
Coefficients represent association between shocks and territorial growth.

In [ ]:
reg = pd.read_csv(reg_path)
reg